# Laboratorio: Recommendation con Python (senza librerie)

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
nome.cognome@unibo.it

## Introduzione a Jupyter

- Questo è un documento Jupyter
- Al suo interno si trovano celle di codice Python eseguibili
- Eseguendo una cella, il risultato che si ottiene è riportato sotto la cella stessa
  - stringhe stampate con `print` e/o risultato di un'espressione
- Le celle di codice possono essere modificate e rieseguite liberamente

- Questo è un esempio di cella di codice:

In [ ]:
20 + 20 + 2

- Se state vedendo questo file in Jupyter, cliccate sulla cella e premete **Maiusc + Invio** (o cliccate "Run" in alto) per eseguirla: il risultato dell'espressione comparirà sotto
  - in _nbviewer_ potete solo visualizzare il file, non potete eseguire codice
- Potete aggiungere una nuova cella di codice sotto a quella corrente cliccando sul pulsante "+" in alto

### Comandi principali da tastiera

- **Ctrl + Invio**: esegui cella corrente
- **Maiusc + Invio**: esegui cella corrente e seleziona la successiva
- **Ctrl + Maiusc + -**: dividi cella corrente in due dove si trova il cursore
- **Maiusc + M**: unisci celle selezionate
  - tenere premuto **Maiusc** per selezionare un intervallo di celle

I comandi sotto funzionano sulla cella selezionata solo se non se ne sta modificando il contenuto:
- **Invio**: modifica contenuto
- **A/B**: crea nuova cella sopra/sotto
- **D, D** (due volte D): elimina cella
- **X/C/V**: taglia/copia/incolla cella
- **Y/M**: converti cella in codice/testo

## Recommendation: Prevedere le Propensioni di Acquisto <br>e Molto Altro ... 

- Ogni azienda ha i dati storici di acquisto di ciascun cliente/utente
  - usiamo un sottoinsieme di dati reali presi da Amazon
- Vogliamo **raccomandare/suggerire ai singoli utenti quali prodotti acquistare** 
  - idea: utenti con storie di acquisti simili, è probabile che faranno acquisti simili anche in futuro 
  - metodo: proponiamo ad ogni utente gli acquisti fatti da altri utenti con storico più simile al proprio
- Vediamo come estrarre i suggerimenti sfruttando le **strutture dati standard di Python** e le operazioni che offrono
  - vedremo poi come farlo in modo ancora più efficiente con operazioni tra matrici...

## Scaricamento File Dati

- Un archivio ZIP con i file necessari per l'esercitazione si trova all'URL https://git.io/fhxQh
- Se state usando Binder, avete già i file nella directory corrente
- Altrimenti, eseguite la seguente cella di codice per
  - verificare se il file ZIP con i dati sia già presente
  - scaricare il file ZIP se non è già presente
  - estrarre i file nella cartella corrente
- Si può vedere come la libreria standard di Python fornisca già funzioni per eseguire agevolmente queste operazioni

In [1]:
import os.path
if not os.path.exists("purchases_data.zip"):
    from urllib.request import urlretrieve
    urlretrieve("https://git.io/fhxQh", "purchases_data.zip")
    from zipfile import ZipFile
    with ZipFile("purchases_data.zip") as f:
        f.extractall()

## Caricamento Nomi Utenti

- Il file `users.csv` contiene un elenco degli utenti coinvolti nell'analisi
  - sono stati selezionati gli utenti con almeno 30 acquisti nello storico
- Si tratta in pratica di un file CSV (_Comma Separated Values_), contenente una riga per ogni utente nel formato `IdUtente;Nome`
- Possiamo usare il modulo `csv` della libreria standard di Python per leggere tali file in modo semplice
- Importiamo il modulo per caricarlo in memoria

In [2]:
import csv

- Per **aprire il file** utilizziamo la funzione `open` specificando il nome del file
  - di default il file è aperto in lettura e modalità testo
- Usiamo il costrutto `with` per **chiudere automaticamente il file** una volta usato
- La funzione `csv.reader` **itera una ad una le righe del file** e restituisce per ciascuna una tupla con le stringhe ID utente ("User ID", `uid`) e nome
  - va specificato che i due sono separati da ";"
- Creiamo un dizionario `users` dove gli ID utente (convertiti in numeri `int`) sono le chiavi e i nomi sono i valori

In [3]:
with open("users.csv", "r") as f:
    users = {int(uid): name for uid, name in csv.reader(f, delimiter=";")}

- Possiamo quindi reperire il nome di un qualsiasi utente dato il suo ID

In [4]:
users[84]

'malachix'

- Tramite la funzione `len` possiamo contare il numero totale di utenti

In [5]:
len(users)

178

## Caricamento Nomi Prodotti

- Il file `items.csv` contiene i prodotti distinti acquistati dagli utenti sopra
- Il formato del file è analogo a quello sopra, con righe `IdProdotto;Nome`
- Ne salviamo il contenuto in un dizionario `items`, ottenuto come fatto sopra con `users`

In [6]:
with open("items.csv", "r") as f:
    items = {int(iid): name for iid, name in csv.reader(f, delimiter=";")}

- Come sopra, possiamo ottenere il nome di qualsiasi prodotto dato l'ID ("Item ID", `iid`)...

In [7]:
items[2669]

'Independence Day [VHS]'

...e contarne il numero totale

In [8]:
len(items)

3384

## Caricamento Dati Acquisti

- Il file CSV `purchases-2000.csv` contiene i dati sugli acquisti effettuati dagli utenti analizzati fino alla fine del 2000
- Per ciascun prodotto acquistato da ogni utente, il file contiene una riga `IdUtente;IdProdotto`
- Usiamo le funzioni viste sopra per leggere il file, creando stavolta un insieme (`set`) di tuple `(uid, iid)`

In [9]:
with open("purchases-2000.csv", "r") as f:
    purchases = {(int(uid), int(iid)) for uid, iid in csv.reader(f, delimiter=";")}

- Quante tuple abbiamo estratto?

In [10]:
len(purchases)

9683

_**Quesito:** qual è il numero di acquisti medio per cliente?_

## Raggruppare gli Acquisti per Utente e per Prodotto

- Per lavorare più agevolmente con questi dati, dall'insieme "globale" di acquisti che abbiamo caricato, ricaviamo
  - un dizionario che associ a ciascun utente l'insieme di prodotti che ha acquistato
  - un dizionario che associ a ciascun prodotto l'insieme di utenti che lo hanno acquistato
- Per agevolare il compito, creiamo prima una funzione `get_purchases_by` che, dato un ID utente, restituisca l'insieme di ID dei prodotti acquistati
  - si iterano (`for...in`) tutte le tuple `(uid, iid)` nell'insieme `purchases`
  - si selezionano (`if`) solo quelle il cui `uid` è quello indicato
  - di queste si prendono solamente l'`iid` e si raccolgono in un insieme (`set`)

In [11]:
def get_purchases_by(target_uid):
    return set(iid for uid, iid in purchases if target_uid == uid)

- Con questa funzione possiamo facilmente creare un dizionario `purchases_by_user` che associ ad ogni UID l'insieme di acquisti dato chiamando la funzione
  - usiamo `users.keys()` per iterare tutti gli ID utente _(`.keys()` non è necessario in quanto si itererebbe comunque sulle chiavi, ma viene incluso per chiarezza)_

In [12]:
purchases_by_user = {uid: get_purchases_by(uid) for uid in users.keys()}

- Possiamo verificare che ogni utente abbia effettivamente almeno 30 acquisti
  - col metodo `values` iteriamo gli insiemi di oggetti acquistati (i soli valori, non le chiavi) nel dizionario `purchases_by_user`
  - con `len` estraiamo il numero di elementi di ciascuno
  - con `min` estraiamo il più piccolo di essi e verifichiamo che sia 30

In [13]:
min(len(itemset) for itemset in purchases_by_user.values())

30

- Creiamo analogamente un dizionario `purchases_by_item`, che associ ad ogni ID oggetto l'insieme di utenti che l'hanno acquistato, dato da una funzione `get_purchases_of` che definiamo

In [14]:
def get_purchases_of(target_iid):
    return set(uid for uid, iid in purchases if target_iid == iid)

In [15]:
purchases_by_item = {iid: get_purchases_of(iid) for iid in items.keys()}

_**Quesito:** quante volte è stato acquistato come minimo ciascuno dei prodotti considerati?_

## Similarità tra Utenti

- Vogliamo suggerire prodotti agli utenti in base a **cos'hanno acquistato utenti simili**
- Come determinare quanto due utenti siano "simili"?
- Possiamo contare **quanti sono i prodotti che entrambi hanno acquistato**
- Per ottenere i prodotti acquistati da entrambi due utenti, possiamo calcolare **l'intersezione** degli insiemi dei prodotti acquistati
- Sugli insiemi si può usare l'operatore `&` (AND) per calcolare l'intersezione
- Ad esempio, gli ID dei prodotti acquistati sia dall'utente con ID 84 che da quello con ID 7661 sono:

In [16]:
purchases_by_user[84] & purchases_by_user[7661]

{5162, 43911, 43921, 100267}

- Creiamo una funzione `similarity` che, dati due ID utente, restituisca il numero di prodotti nell'intersezione dei loro acquisti

In [17]:
def similarity(uid1, uid2):
    return len(purchases_by_user[uid1] & purchases_by_user[uid2])

- Usiamo questa funzione per creare un dizionario `similiarities` che, ad ogni tupla con due ID utente, associa la loro similarità
  - scorriamo tutti gli ID utente attraverso due cicli (`for`) innestati, eliminando le coppie di ID uguali
  - _(per semplicità, lasciamo che la similarità di ogni coppia sia calcolata due volte)_

In [18]:
similarities = {(i, j): similarity(i, j) for i in users.keys() for j in users.keys() if i != j}

- Ad esempio, in base all'esempio sopra, la similarità tra gli utenti 84 e 7661 deve essere 4

In [19]:
similarities[(84, 7661)]

4

## Stimare il Potenziale Interesse nei Prodotti

- Vogliamo stimare **quanto ciascun utente sia potenzialmente interessato** in ciascun prodotto non ancora acquistato
- Possiamo stimarlo in base a quanto il prodotto **sia stato acquistato da utenti simili**
- Associamo per ogni utente U e prodotto I un _punteggio d'interesse_ pari alla somma delle similarità degli altri utenti che hanno acquistato il prodotto
- Creiamo una funzione `interest` che calcoli tale punteggio per un utente e un prodotto dati
  - vengono iterati gli utenti che hanno acquistato I, escludendo però U
  - tramite `sum` viene fatta la somma delle similarità

In [20]:
def interest(uid, iid):
    return sum(similarities[(uid, ouid)] for ouid in purchases_by_item[iid] if uid != ouid)

- Raccogliamo tutti i punteggi in un dizionario che associa ad ogni utente U un dizionario di punteggi d'interesse
  - ciascuno associa a sua volta a ciascun prodotto il punteggio d'interesse
  - sono però esclusi i prodotti già acquistati da U

In [21]:
interests_by_user = {
    uid: {
        iid: interest(uid, iid)
        for iid in items.keys()
        if iid not in purchases_by_user[uid]
    } for uid in users.keys()
}

## Ottenere _N_ Suggerimenti di Acquisto per ogni Utente

- Da migliaia di prodotti nel catalogo, vogliamo suggerirne **un numero limitato ad ogni utente** massimizzando la probabilità di acquisto
- Fissiamo un numero _N_ di prodotti da suggerire...

In [22]:
N = 20

- ...vogliamo selezionare per ogni utente gli **_N_ prodotti con "potenziale interesse" maggiore**

- Creiamo una funzione `suggest` che, dato un utente, restituisca _N_ oggetti suggeriti in base ai punteggi d'interesse calcolati sopra
  1. da `interests_by_user` vengono ottenute le tuple `(IdOggetto, Punteggio)` per l'utente selezionato
  2. tali tuple sono ordinate per punteggio decrescente (si noti il meno)
  3. viene restituito l'insieme degli ID oggetti delle prime _N_ tuple

In [23]:
def suggest(uid):
    interests = interests_by_user[uid].items()                # 1
    sorted_interests = sorted(interests, key=lambda x: -x[1]) # 2
    return set(iid for iid, score in sorted_interests[:N])    # 3

- Applichiamo la funzione data a tutti gli utenti

In [24]:
suggestions_by_user = {uid: suggest(uid) for uid in users.keys()}

- Abbiamo così per ciascun utente un set di _N_ prodotti non precedentemente acquistati da suggerire

In [25]:
print(suggestions_by_user[84])

{44037, 60041, 96025, 43290, 57372, 5288, 59817, 7985, 7989, 43586, 60230, 96454, 96456, 95843, 57190, 2669, 101103, 56561, 95480, 44030}


- Ad esempio, per l'utente 84, stampiamo i titoli dei film che ha acquistato...
  - sostituire `"; "` con `"\n"` per visualizzare i titoli uno sotto l'altro

In [26]:
print("; ".join(items[iid] for iid in purchases_by_user[84]))

Godzilla [VHS]; The Lion King [VHS]; Eyes Wide Shut [VHS]; Batman & Robin [VHS]; The Matrix; Blade [VHS]; First Knight [VHS]; Omen 3: The Final Conflict [VHS]; Summer of Sam [VHS]; The World Is Not Enough [VHS]; The Chinese Connection [VHS]; Enter the Dragon [VHS]; Star Wars - Episode I, The Phantom Menace [VHS]; Fists of Fury [VHS]; Alien [VHS]; Batman Returns (1992); Beloved; Total Recall; Touch of Evil [VHS]; Batman Forever; Inspector Gadget; Enemy of the State; Blade Runner (The Director's Cut); Jurassic Park (Widescreen Edition) [VHS]; Tomorrow Never Dies (Limited Edition Gift Pack) [VHS]; Aliens [VHS]; A Bug's Life; Return of the Dragon [VHS]; Batman (1989); The Exorcist; Lost in Space [VHS]; GoldenEye (Special Edition); Excalibur [VHS]; Alien Resurrection [VHS]; Alien 3 [VHS]; Reservoir Dogs [VHS]; Game of Death [VHS]; Lost World: Jurassic Park [VHS]


- ...e i titoli dei film suggeriti

In [27]:
print("; ".join(items[iid] for iid in suggestions_by_user[84]))

Double Jeopardy; Deep Blue Sea; American Pie - Rated Edition (Special Edition) [VHS]; The Green Mile [VHS]; The Sixth Sense [VHS]; Being John Malkovich; Saving Private Ryan [VHS]; The Insider; Fight Club; Galaxy Quest [VHS]; Three Kings; The Talented Mr. Ripley; Titanic [VHS]; Jaws [VHS]; Curse of the Blair Witch [VHS]; Independence Day [VHS]; Abyss [VHS]; Armageddon [VHS]; Dogma [VHS]; Sleepy Hollow


## Accuratezza dei Suggerimenti di Acquisto

- Come valutare se i suggerimenti ottenuti in questo modo siano azzeccati?
- Una possibilità consiste nel verificare **se gli oggetti suggeriti siano stati effettivamente acquistati** in un successivo momento
- Nel file `purchases-2014.csv` è fornita una seconda lista di acquisti aggiornata che include anche quelli successivi al 2000
  - utenti e prodotti sono limitati a quelli già caricati in `users` e `items`
- Possiamo quindi confrontare i prodotti suggeriti con questa nuova matrice

In [28]:
with open("purchases-2014.csv", "r") as f:
    purchases_updated = {(int(uid), int(iid)) for uid, iid in csv.reader(f, delimiter=";")}

### Selezionare solo i Nuovi Acquisti

- Il nuovo file riporta **tutti** gli acquisti, compresi quelli già indicati nel file precedente
- Vogliamo un insieme dei soli acquisti successivi all'analisi svolta sopra
- Possiamo ottenerlo calcolando la differenza tra gli insiemi di acquisti, tramite l'operatore `-`

In [29]:
new_purchases = purchases_updated - purchases

- Possiamo raggruppare questi nuovi acquisti per utente, in modo analogo a quanto fatto sopra

In [30]:
def get_new_purchases_by(target_uid):
    return set(iid for uid, iid in new_purchases if target_uid == uid)

new_purchases_by_user = {uid: get_new_purchases_by(uid) for uid in users.keys()}

### Quali Nuovi Acquisti sono stati Suggeriti?

- Abbiamo ora il dizionario `suggestions_by_user` con gli acquisti _suggeriti_ e `new_purchases_by_user` con i nuovi acquisti _effettivi_
- Da questi possiamo individuare quali sono i suggerimenti **validi**, quelli a cui dopo l'analisi è corrisposto un acquisto
- Consideriamo un utente _soddisfatto_ se ha ricevuto **almeno un suggerimento valido**
- Individuiamo l'insieme degli utenti soddisfatti individuando quelli dove l'intersezione tra suggerimenti e nuovi acquisti non è vuota
  - ricordare che usando un insieme (o un'altra collezione) in `if`, otteniamo `True` se e solo se l'insieme non è vuoto

In [31]:
satisfied_users = {uid for uid in users.keys() if suggestions_by_user[uid] & new_purchases_by_user[uid]}

- Quanti sono gli utenti soddisfatti?

In [32]:
len(satisfied_users)

62

- Quanti sono come percentuale rispetto al totale degli utenti analizzati?

In [33]:
len(satisfied_users) / len(users)

0.34831460674157305

- Abbiamo quindi suggerito **almeno un prodotto valido** per circa **un terzo degli utenti**

## Esercizi: Confronto con una Selezione Casuale di Prodotti

- Per valutare quanto il risultato ottenuto sia buono, possiamo misurare cosa otterremmo **suggerendo _N_ prodotti a caso** a ciascun utente
- Per generare numeri casuali, usiamo il modulo `random` di Python

In [34]:
import random

- Per ottenere risultati riproducibili, impostiamo un valore fisso come seed

In [35]:
random.seed(1234567)

- **1)** creare una funzione `suggest_random` che, dato un ID utente, restituisca una lista di _N_ ID prodotti casuali tra quelli che non risultano da lui acquistati nel dataset del 2000
  - per un utente `uid`, gli ID dei prodotti non acquistati sono le chiavi di `interests_by_user[uid]`
  - la funzione `sample(x, k)` del modulo `random` seleziona una lista di k elementi casuali da x
  - tale lista va convertita in un `set` per compatibilità con i passaggi successivi

In [36]:
def suggest_random(uid):
    ...

- **2)** usare la funzione per creare un dizionario `random_suggestions_by_user` che associ ad ogni utente i suoi suggerimenti casuali
  - come riferimento si usi la creazione del dizionario `suggestions_by_user` sopra

In [37]:
random_suggestions_by_user = ...

- **3)** creare un insieme `randomly_satisfied_users` con gli ID degli utenti per cui almeno un prodotto tra i suggerimenti casuali è stato acquistato in seguito
  - usare `satisfied_users` come riferimento

In [38]:
randomly_satisfied_users = ...

- **4)** calcolare la percentuale di utenti in `randomly_satisfied_users` rispetto al totale
  - tale percentuale dovrebbe risultare intorno al **12%**
- **Extra.** Questa percentuale può cambiare variando il valore `seed` in alto. Rieseguire i calcoli sopra con 2-3 seed differenti, quindi calcolare la percentuale media su 10000 prove con seed diversi.

## Sviluppi Successivi

- Abbiamo quì visto come usare le strutture dati e le funzioni standard di Python per un compito pratico
- Nel prossimo laboratorio vedremo come ottenere lo stesso risultato tramite **operazioni tra matrici e algebra lineare**